In [13]:
# An attempt to recreate studies/cuda-kernels/reducers/awkward_reduce_sum_atomics.py using cccl instead of raw cuda kernels
import awkward as ak
import cupy as cp
import numpy as np

from cuda.compute import segmented_reduce

def sum_op(a, b):
    return a+b

awkward_array = ak.Array([[1], [2, 3], [4, 5], [6, 7, 8], [], [9]], backend = 'cuda')
input_data = awkward_array.layout.content.data 
offsets = awkward_array.layout.offsets.data

# Prepare the start and end offsets
start_o = offsets[:-1]
end_o = offsets[1:]

# Prepare the output array
n_segments = start_o.size
output = cp.empty(n_segments, dtype=cp.int32)

# Initial value for the reduction
h_init = np.array([0], dtype=np.int32)

# Perform the segmented reduce
segmented_reduce(
    input_data, output, start_o, end_o, sum_op, h_init, n_segments
)

print(f"Segmented reduce result: {output.get()}")

Segmented reduce result: [ 1  5  9 21  0  9]


Success!

In [7]:
# Verify the result.
expected_output = cp.asarray([1, 5, 9, 21, 0, 9], dtype=output.dtype)
assert (output == expected_output).all()

### Compare time metrics between cuda-kernels and cccl

Using cuda-kernels:

In [17]:
import cupy as cp

cuda_kernel = """
extern "C" {
    __global__ void awkward_reduce_sum_a(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;

       if (thread_id < outlength) {
          toptr[thread_id] = 0;
       }
    }
}
extern "C" {
    __global__ void awkward_reduce_sum_b(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
       int stride = blockDim.x * gridDim.x;

       for (int i = thread_id; i < lenparents; i += stride) {
           atomicAdd(&toptr[parents[i]], fromptr[i]);
       }
    }
}
"""

In [18]:
%%timeit
parents = cp.array([0, 1, 1, 2, 2, 3, 3, 3, 5], dtype=cp.int32)
fromptr = cp.array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=cp.int32)
lenparents = len(parents)
outlength = int(cp.max(parents)) + 1
toptr = cp.zeros(outlength, dtype=cp.int32)

block_size = 256
grid_size = (lenparents + block_size - 1) // block_size

raw_module = cp.RawModule(code=cuda_kernel)

awkward_reduce_sum_a = raw_module.get_function('awkward_reduce_sum_a')
awkward_reduce_sum_b = raw_module.get_function('awkward_reduce_sum_b')

awkward_reduce_sum_a((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))
awkward_reduce_sum_b((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))

788 μs ± 41.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Using cccl:

In [23]:
# An attempt to recreate studies/cuda-kernels/reducers/awkward_reduce_sum_atomics.py using cccl instead of raw cuda kernels

import awkward as ak
import cupy as cp
import numpy as np

from cuda.compute import segmented_reduce

def sum_op(a, b):
    return a+b

In [28]:
%%timeit
awkward_array = ak.Array([[1], [2, 3], [4, 5], [6, 7, 8], [], [9]], backend = 'cuda')
input_data = awkward_array.layout.content.data 
offsets = awkward_array.layout.offsets.data

# Prepare the start and end offsets
start_o = offsets[:-1]
end_o = offsets[1:]

# Prepare the output array
n_segments = start_o.size
output = cp.empty(n_segments, dtype=np.int32)

# Initial value for the reduction
h_init = np.array([0], dtype=np.int32)

# Perform the segmented reduce
segmented_reduce(
    input_data, output, start_o, end_o, sum_op, h_init, n_segments
)

489 μs ± 50.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
